In [157]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import sympy
import numpy as np
from matplotlib import pyplot as plt


P, a, k, Q = sympy.symbols("P a k Q")
p, q = sympy.symbols("p q")

N, w = sympy.symbols("N w")

insideSum = sympy.binomial(P,a)*p**a*(1-p)**(P-a)*sympy.binomial(Q, k-a)*q**(k-a)*(1-q)**(Q-(k-a))
sumTerm = sympy.Sum(insideSum
                    , [a, 0, k]) # this includes lower and upper limit


print("Note that qv is a percent")

def computePr(Pv, Qv, qv, kv):
    # note that with qv=0.0 or 1.0, you'll see it try to simplify 0**anything = 0. This is wrong, since power could be zero.
    print((P, Pv, Q, Qv, q, qv, k, kv))
    qv = sympy.Rational(qv, 100) # we represent q as a percent, this prevents numerical instability
    subsDict = {P: Pv, Q: Qv, q: qv, k: kv}
    
    
    baseFormula = sumTerm.subs(subsDict)
    baseFormula = baseFormula.doit()
    fig = plt.figure()
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1) 
    xSteps = 20
    X = np.linspace(0, 1, xSteps)
    YPr = [baseFormula.subs({p: pv}).evalf() for pv in X]
    ax1.plot(X, YPr)
    plt.xlabel("p")
    
    SimplerPr = []
    YVar = []
    # there's a numerical instability if we try and compute it right out
    # so instead we will do it seperately for each p
    for i in range(xSteps):
        pv = sympy.Rational(i, xSteps-1)
        varSubsDict = {P: Pv, Q: Qv, q: qv, p: pv}
        var = 0
        insideSumAfterSubs = insideSum.subs(varSubsDict)
        #print("inside sum:", insideSum)
        #print("inside sum after subs:", insideSumAfterSubs)
        
         # (sum over k of Pr(heads=k)*(k/(Q+P))**2)    -  mu**2, where mu=(P*p+Q*q)/(P+Q)
        #print("curP:", pv)
        for kv in range(0, (Q+P).subs(varSubsDict)+1): # +1 because range is exclusive
            curPrK = 0
            for av in range(0, kv+1):
                # this order of substitution is important, otherwise it replaces binomial with factorial
                # this sum relies on binomial(a, -b) = 0 (when b is positive), but when converting to factorial
                # that givevs you factorial(negative number) which is -oo, and sympy has trouble dealing with that so
                # you get nan. This substitution order prevents this issue from happening
                valOfTerm = insideSum.subs({k: kv, a: av}).subs(varSubsDict) 
                #print("a=", av, " is ", valOfTerm)
                curPrK += valOfTerm
            #print("pr(k) with k=", kv, " is ", curPrK)
            curPrKTimesX = (curPrK*(k/(Q+P))**2).subs(varSubsDict).subs({k: kv})
            #print(curPrKTimesX)
            var += curPrKTimesX
        mu = ((P*p+Q*q)/(P+Q)).subs(varSubsDict)
        var = var - mu**2
        #print("resVar", var)
        YVar.append(var)
        
        
        
        avgPr = ((q*Q+p*P)/(Q+P)).subs(varSubsDict)
        print(avgPr)
        prK = sympy.binomial(N,kv)*p**kv*(1-p)**(N-kv)
        prK = prK.subs({p: avgPr, N: (Qv+Pv)})
        SimplerPr.append(prK)
        
        '''
        termInner = varTermInner.subs(varSubsDict)
        print("term inner", termInner)
       
        varTerm = manualSum(termInner, [k,0,Q+P]).subs(varSubsDict)
        print("totes", varTerm)
        varTerm = varTerm.doit()
        print("totes", varTerm)
        varTerm = varTerm -((P*p+Q*q)/(P+Q))**2
        varFormula = varTerm.subs(varSubsDict).doit()
        YVar.append(varFormula.evalf())
        '''
    ax2.plot(X, YVar)
    ax3.plot(X, SimplerPr)
    plt.show()
    
updateWidget = interact_manual(computePr, Pv=widgets.IntSlider(min=0, max=30, step=1, value=2),
          Qv=widgets.IntSlider(min=0, max=30, step=1, value=2),
          qv=widgets.IntSlider(min=0, max=100, step=1, value=50),
          kv=widgets.IntSlider(min=0, max=60, step=1, value=2))
updateWidget

Note that qv is a percent


interactive(children=(IntSlider(value=2, description='Pv', max=30), IntSlider(value=2, description='Qv', max=3…

<function __main__.computePr(Pv, Qv, qv, kv)>

In [11]:
0.0**0


1.0

In [18]:
sympy.binomial(10, 0)


1

In [20]:
(0.0**(5 - a)*p**a*(1 - p)**(10 - a)*sympy.binomial(10, a)*sympy.binomial(10, 5 - a)).subs({a: 5})

252*p**5*(1 - p)**5

In [23]:
sympy.Sum(a, (a, 1, 4)).doit()

10

In [93]:
sympy.binomial(-2, -2)


0

In [122]:
(p**(P)*q**(Q)).subs({p: 0})

0**P*q**Q

In [120]:
0**0

1

In [131]:
sympy.binomial(2, -1)

0